In [1]:
import sagemaker
import boto3
import xgboost as xgb
from sagemaker import get_execution_role
from sagemaker.inputs import TrainingInput
from sagemaker.xgboost import XGBoostModel

/opt/conda/lib/python3.11/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [2]:
sess = sagemaker.Session()
bucket = sess.default_bucket()
role = get_execution_role()

In [3]:
s3_bucket = "ads-508-final"
s3_prefix = "transformeddata"
file_path_train = f"s3://{s3_bucket}/{s3_prefix}/sm_fm_train.libsvm"
file_path_test = f"s3://{s3_bucket}/{s3_prefix}/sm_fm_test.libsvm"

In [4]:
#Train and Test Data
train_data = TrainingInput(file_path_train, content_type="libsvm")
test_data = TrainingInput(file_path_test, content_type="libsvm")

In [9]:
#Parameters
hyperparameters = {
    "objective": "binary:logistic",
    "max_depth": 6, 
    "eta": 0.1,  
    "gamma": 0,  
    "min_child_weight": 1,  
    "eval_metric": "logloss", 
    "num_round": 100, 
}

In [11]:
#Estimator 
from sagemaker.estimator import Estimator

xgboost_estimator = Estimator(
    image_uri=sagemaker.image_uris.retrieve("xgboost", region=sess.boto_region_name, version="1.0-1"),
    role=role,
    instance_count=1,
    instance_type="ml.m5.large",
    hyperparameters=hyperparameters,
    output_path=f"s3://{s3_bucket}/{s3_prefix}/output",
    base_job_name="xgboost-recommender",
)

[04/01/25 10:58:52] INFO     Defaulting to only available Python version: py3                     ]8;id=253145;file:///opt/conda/lib/python3.11/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=65559;file:///opt/conda/lib/python3.11/site-packages/sagemaker/image_uris.py#610\610]8;;\

                    INFO     Defaulting to only supported image scope: cpu.                       ]8;id=783052;file:///opt/conda/lib/python3.11/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=824074;file:///opt/conda/lib/python3.11/site-packages/sagemaker/image_uris.py#534\534]8;;\

In [12]:
#Fit
xgboost_estimator.fit({"train": train_data, "validation": test_data})

[04/01/25 10:59:04] INFO     SageMaker Python SDK will collect telemetry to help us better  ]8;id=617394;file:///opt/conda/lib/python3.11/site-packages/sagemaker/telemetry/telemetry_logging.py\telemetry_logging.py]8;;\:]8;id=474556;file:///opt/conda/lib/python3.11/site-packages/sagemaker/telemetry/telemetry_logging.py#91\91]8;;\
                             understand our user's needs, diagnose issues, and deliver                             
                             additional features.                                                                  
                             To opt out of telemetry, please disable via TelemetryOptOut                           
                             parameter in SDK defaults config. For more information, refer                         
                             to                                                                                    
                             https://sagemaker.readthedocs.io/en/stable/overview.html#confi                        
                             guring-and-using-defaults-with-the-sagemaker-python-sdk.                              

                    INFO     Creating training-job with name:                                       ]8;id=866869;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=84385;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py#1042\1042]8;;\
                             xgboost-recommender-2025-04-01-10-59-04-330                                           

2025-04-01 10:59:05 Starting - Starting the training job...
..25-04-01 10:59:21 Starting - Preparing the instances for training.
..25-04-01 10:59:42 Downloading - Downloading input data.
....[2025-04-01 11:01:18.044 ip-10-0-222-72.ec2.internal:7 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:Failed to parse hyperparameter eval_metric value logloss to Json.
Returning the value itself
INFO:sagemaker-containers:Failed to parse hyperparameter objective value binary:logistic to Json.
Returning the value itself
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Running XGBoost Sagemaker in algorithm mode
INFO:root:Single node training.
[11:01:19] 7140627x3 matrix with 14281254 entries loaded from /opt/ml/input/data/train
[11:01:19] 1785157x3 matrix with 3570314 entries loaded from /opt/ml/input/data/validation
[2025

In [ ]:
#Predictions

xgboost_model = xgboost_estimator.deploy(
    initial_instance_count=1, 
    instance_type="ml.m5.large"
)